In [267]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from functools import partial
import matplotlib.pyplot as plt
import pandas as pd
import jax.random as random
import numpyro
import numpyro.distributions as dist
import jax.numpy as jnp
import numpy as np

import tarea

rng_key = random.PRNGKey(12345)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# ¿Es posible explicar la cantidad de billonarios en base al desarrollo país?  <a class="tocSkip"></a>







En 2006 *Daniel Treisman* publicó un artículo titulado [*Russia Billionaries*](https://pubs.aeaweb.org/doi/pdfplus/10.1257/aer.p20161068) en el cual conectó la cantidad de billonarios de un país con ciertos atributos económicos de los mismos. 

Su conclusión principal fue que Rusia tiene una cantidad de billonarios mayor que la que predicen los indicadores económicos

En esta tarea ustedes analizarán datos macroeconómicos para comprobar o refutar los hallazgos de *D. Treisman*

## Datos

Para esta tarea se les provee de un conjunto de datos `billonarios.csv` indexado por país con los siguientes atributos

- `nbillonarios`: La cantidad de billonarios del pais
- `logpibpc`: El logaritmo del Producto Interno Bruto (PIB) per capita del pais
- `logpob`: El logaritmo de la población del pais
- `gatt`: La cantidad de años que el pais está adherido al *General Agreement on Tariffs and Trade* (GATT)

In [357]:
df = pd.read_csv('data/billonarios.csv', index_col='pais')
display(df.head(5))
yb = df["nbillonarios"].values
xb  = df.drop("nbillonarios", axis=1).values
display(yb.shape, xb.shape)

,nbillonarios,logpibpc,logpob,gatt
pais,,,,
United States,469,10.786021,19.532846,60
Canada,25,10.743365,17.319439,0
"Bahamas, The",0,10.072139,12.760934,0
Aruba,0,10.223734,11.526276,0
Bermuda,0,11.446847,11.086334,0


(197,)

(197, 3)

## Modelo 

El objetivo principal de esta tarea es entrenar un modelo de regresión que prediga la cantidad de billonarios en función de los demás atributos

> El número de billonarios es una variable entera y no-negativa. 

Un modelo apropiado en este caso es la [regresión de Poisson](https://en.wikipedia.org/wiki/Poisson_distribution), donde definimos la probabilidad condicional para un pais $i$ como  

$$
p(y_i | x_i ) = \frac{\lambda_i^{y_i}}{y_i!} \exp \left ({-\lambda_i} \right)
$$

con intensidad

$$
\lambda_i = \exp \left (\theta_0 + \sum_{j=1}^M \theta_j x_{ij} \right)
$$

donde 

- $\theta$ es el vector de parámetros que deseamos ajustar 
- $y_i$ y $x_i$ son la cantidad de billonarios y el vector de atributos del país $i$, respectivamente


## Actividades

- Implemente el modelo usando numpyro (función `model` en `tarea.py`)
    - Considere un prior normal para los parámetros $\theta$
    - Considere una verosimilitud poisson
- Obtenga muestras del posterior utilizando MCMC
    - Muestre las trazas de los parámetros (utilize `matplotlib`)
    - Diagnóstique la convergencia en base a las trazas, número de muestras efectivo y el estadístico de Gelman-Rubin
- Análisis los resultados obtenidos
    - Muestre el posterior de los parámetros obtenidos (utilize `matplotlib`), ¿Cúales son significativamente distintos de cero?
    - Prediga la cantidad de billonarios y la incertidumbre asociada de cada pais usando su modelo (posterior predictivo. 
    - Responda y discuta ¿Cuáles son los 5 países con mayor error entre la predicción y el valor real? ¿Cuáles países tienen un exceso de billonarios? ¿Cúales paises tienen menos billonarios de lo esperado? ¿Qué puede decir sobre Rusia? ¿Cuáles son los 5 paises donde el modelo está más inseguro de sus resultados?

In [338]:
seeded_model = numpyro.handlers.seed(tarea.model, random.PRNGKey(12345))
exec_trace = numpyro.handlers.trace(seeded_model).get_trace(x, y)
print(numpyro.util.format_shapes(exec_trace))

(197,)
(197,)
(3,)
Trace Shapes:      
 Param Sites:      
Sample Sites:      
  theta0 dist   3 |
        value     |
  theta1 dist   3 |
        value     |
  theta2 dist   3 |
        value     |
  theta3 dist   3 |
        value     |
  datos plate 197 |


In [391]:
# inicializacion modelo generativo

np.random.seed(1234)

t0,t1,t2,t3, N = 0.2, 0.3, 0.2, 0.1, 197
#x = [np.random.randn(N), np.random.randn(N), np.random.randn(N)]
x = xb.T
y = np.exp(t0 + t1*x[0] + t2*x[1] + t3*x[2])
x_test = [np.random.randn(N), np.random.randn(N), np.random.randn(N)]
# genero 3 semillas


# Creamos el objetivo Predictive
predictive = numpyro.infer.Predictive(tarea.model, 
                                      num_samples=2000)

# Para muestrear el objeto predictive requiere una semilla aleatoria y las variables de entrada del modelo
rng_key = random.PRNGKey(12345)
rng_key, rngkey = random.split(rng_key)
prior_samples = predictive(rngkey, x_test)
prior_samples.keys()


# muestras del posterior con MCMC

rng_key, rngkey = random.split(rng_key)

sampler = numpyro.infer.MCMC(sampler=numpyro.infer.NUTS(tarea.model), 
                             num_samples=1000, num_warmup=100, thinning=1,
                             num_chains=2)

sampler.run(rngkey, x, y)



sampler.print_summary(prob=0.9)

/home/marco/anaconda3/envs/info274/lib/python3.10/site-packages/numpyro/infer/mcmc.py:274: UserWarning: There are not enough devices to run parallel chains: expected 2 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(2)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  warnings.warn(
sample: 100%|█████████████████████████| 1100/1100 [00:01<00:00, 658.63it/s, 1023 steps of size 6.31e-09. acc. prob=0.85]



                mean       std    median      5.0%     95.0%     n_eff     r_hat
         s      0.49      0.32      0.49      0.17      0.81      1.00 2910923.90
  theta[0]     -0.42      1.46     -0.42     -1.88      1.04      1.00 401506.00
  theta[1]      1.07      0.56      1.07      0.52      1.63      1.00  93880.33
  theta[2]     -0.98      0.13     -0.98     -1.11     -0.84      1.00  27602.96
  theta[3]     -0.19      0.37     -0.19     -0.56      0.17      1.00 294985.88

Number of divergences: 0


In [390]:
rng_key, rngkey = random.split(rng_key)

sampler1 = numpyro.infer.MCMC(sampler=numpyro.infer.BarkerMH(tarea.model), 
                             num_samples=1000, num_warmup=100, thinning=1,
                             num_chains=2)

sampler1.run(rngkey, x, y)



sampler1.print_summary(prob=0.9)

/home/marco/anaconda3/envs/info274/lib/python3.10/site-packages/numpyro/infer/mcmc.py:274: UserWarning: There are not enough devices to run parallel chains: expected 2 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(2)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  warnings.warn(
sample: 100%|█████████████████████████████████| 1100/1100 [00:00<00:00, 8372.73it/s, step size 4.62e-03. acc. prob=0.14]


                mean       std    median      5.0%     95.0%     n_eff     r_hat
         s      0.65      0.11      0.65      0.54      0.76      1.00    168.42
  theta[0]     -1.26      0.89     -1.22     -2.18     -0.38      1.00     60.65
  theta[1]      0.51      0.78      0.50     -0.27      1.32      1.00     91.59
  theta[2]     -0.93      0.80     -0.93     -1.73     -0.12      1.00    169.91
  theta[3]     -0.13      0.17     -0.14     -0.30      0.05      1.00     54.34



In [382]:
for i in range(3): # Tres semillas
    seeded_model = numpyro.handlers.seed(model, random.PRNGKey(i))
    print(i, seeded_model(x))

0 [           inf 7.59614203e-24 8.80854708e-18 3.04545640e-16
 4.65717773e-16 5.51669359e-08            inf            inf
            inf            inf            inf 1.86478887e+11
 9.91681408e+08 2.47273120e+08 5.34230180e+10 1.30289728e+21
            inf            inf 6.45775648e+26 1.67881728e+08
 9.52633789e+03 1.54175770e+09            inf 1.06848903e-01
            inf 5.58261958e+23 3.95375040e+08            inf
            inf 2.48505071e-01 9.40816193e+01            inf
 3.42860104e+10 1.10593906e+04            inf            inf
            inf 1.67667927e-15            inf            inf
 4.21349710e-16            inf            inf            inf
 1.64858922e-15 5.60598480e+07            inf            inf
 4.00139052e-16            inf            inf 1.32209689e+27
            inf            inf            inf            inf
            inf 5.89689261e-15 4.57256000e+08 3.49863717e-06
 3.58492546e+01 2.63509330e-21 1.01162738e+26            inf
 2.23145880e-20 3.0966

In [385]:
exec_trace = numpyro.handlers.trace(seeded_model).get_trace(x,y)
print(numpyro.util.format_shapes(exec_trace))

Trace Shapes:        
 Param Sites:        
Sample Sites:        
   theta dist     | 4
        value     | 4
       s dist     |  
        value     |  
  datos plate 197 |  
       y dist 197 |  
        value 197 |  


In [386]:
print(exec_trace)

OrderedDict([('theta', {'type': 'sample', 'name': 'theta', 'fn': <numpyro.distributions.distribution.Independent object at 0x7fcaa2efb040>, 'args': (), 'kwargs': {'rng_key': array([2613193937,  691811511], dtype=uint32), 'sample_shape': ()}, 'value': DeviceArray([-3.3883095, -4.794694 , -2.2919903, -2.470184 ], dtype=float32), 'scale': None, 'is_observed': False, 'intermediates': [], 'cond_indep_stack': [], 'infer': {}}), ('s', {'type': 'sample', 'name': 's', 'fn': <numpyro.distributions.continuous.HalfCauchy object at 0x7fcaa2efa890>, 'args': (), 'kwargs': {'rng_key': array([2409338849, 2235337890], dtype=uint32), 'sample_shape': ()}, 'value': DeviceArray(4.013031, dtype=float32), 'scale': None, 'is_observed': False, 'intermediates': [], 'cond_indep_stack': [], 'infer': {}}), ('datos', {'type': 'plate', 'fn': <function _subsample_fn at 0x7fcbd5ef2710>, 'name': 'datos', 'args': (197, None), 'kwargs': {'rng_key': None}, 'value': DeviceArray([  0,   1,   2,   3,   4,   5,   6,   7,   8, 

In [392]:
# Posterior predictivo, ejemplos y estadísticos
rng_key, rng_key_ = random.split(rng_key)
predictive_samples = tarea.get_predictive_samples(seeded_model, x_test, rng_key_, 
                                                     mcmc_trace=sampler)

fig, ax = plt.subplots(1, 3, figsize=(12, 3), tight_layout=True)   
for i in range(3):
    proyecto.plot_predictive_posterior(ax[i], x1, x2, predictive_samples['y'][i], cmap=binary_cmap)
    proyecto.plot_data(ax[i], x, y)
    
fig, ax = plt.subplots(1, 2, figsize=(12, 4), tight_layout=True)
proyecto.plot_predictive_posterior(ax[0], x1, x2, proyecto.binary_mode(predictive_samples['y']),
                                   title='Moda del posterior predictivo', cmap=binary_cmap)
proyecto.plot_data(ax[0], x, y)
proyecto.plot_predictive_posterior(ax[1], x1, x2, proyecto.entropy(predictive_samples['y']), vmin=0, vmax=1/2, 
                                   title='Entropía del posterior predictivo', cmap=plt.cm.hot)
proyecto.plot_data(ax[1], x, y)

AttributeError: 'MCMC' object has no attribute 'items'